In [1]:
### GROUP 2 ###

# B.J.T. Donkers - 1506897
# S. van der Leek - 1564226
# L. Sloot - 1365711
# S. Diemel  - 1645749
# A.E. Murariu - 1532081
# Z.V. Petrov - 1554999

In [16]:
### IMPORT PACKAGES ###

import pandas as pd
import numpy as np

# process mining for python
import pm4py

# machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# nice plots & tables
import plotly.express as px
import plotly.graph_objects as go

# to make a counter object
from collections import Counter

In [3]:
### IMPORT CSV FUNCTION ###

def import_csv(file_path):
    event_log = pd.read_csv(file_path, sep=',')
    # make a timestamp out of the timefield
    event_log["event time:timestamp"] = pd.to_datetime(event_log["event time:timestamp"])

    # replace nan with 0:
    event_log = event_log.fillna(0)

    # change format into pm4py event_log
    event_log = pm4py.format_dataframe(event_log, case_id = 'case concept:name', activity_key = 'event concept:name',
                                       timestamp_key = "event time:timestamp", timest_format = '%Y-%m-%d %H:%M:%Sz')

    #2011-10-01T00:38:44.546+02:00
    # print what the start and end activities are:
    start_activities = pm4py.get_start_activities(event_log)
    end_activities = pm4py.get_end_activities(event_log)
    print("Start activities: {}\n\nEnd activities: {}".format(start_activities, end_activities))

    # print how many events and how many instances (cases) there are
    num_events = len(event_log)
    num_cases = len(event_log['case concept:name'].unique())
    print(' ')
    print("Number of events: {}\nNumber of cases: {}".format(num_events, num_cases))


    return event_log

In [4]:
### IMPORT TRAIN DATA ###

file = "data/added_features_event_log_2.csv"
event_log = import_csv(file)

Start activities: {'A_SUBMITTED': 10469}

End activities: {'A_DECLINED': 2690, 'W_Valideren aanvraag': 1953, 'W_Afhandelen leads': 1864, 'W_Completeren aanvraag': 1705, 'W_Nabellen offertes': 1286, 'A_CANCELLED': 437, 'W_Nabellen incomplete dossiers': 318, 'O_CANCELLED': 164, 'W_Beoordelen fraude': 49, 'W_Wijzigen contractgegevens': 2, 'A_ACTIVATED': 1}
 
Number of events: 199668
Number of cases: 10469


In [5]:
### IMPORT TEST DATA ###

file = "data/added_features_event_log_test.csv"
test_event_log = import_csv(file)

Start activities: {'A_SUBMITTED': 1938, 'W_Nabellen offertes': 345, 'W_Completeren aanvraag': 197, 'W_Valideren aanvraag': 59, 'W_Nabellen incomplete dossiers': 44, 'W_Afhandelen leads': 33, 'W_Beoordelen fraude': 2}

End activities: {'A_DECLINED': 739, 'W_Completeren aanvraag': 570, 'W_Nabellen offertes': 523, 'W_Afhandelen leads': 412, 'W_Valideren aanvraag': 233, 'W_Nabellen incomplete dossiers': 94, 'A_CANCELLED': 23, 'O_CANCELLED': 15, 'W_Beoordelen fraude': 9}
 
Number of events: 47823
Number of cases: 2618


In [6]:
### BASELINE PREDICTION OF THE NEXT STEP ###

# If you don't know anything about the past, you would always predict the most common event.

my_counter = Counter(event_log['event concept:name'])
print(my_counter)
print(' ')
print('The most common event is: ', my_counter.most_common(1)[0][0], ' this occurs ', my_counter.most_common(1)[0][1], ' times. This is ', round(((my_counter.most_common(1)[0][1])/len(event_log))*100, 2), '% of the events.')
print(' ')
print('Therefore the baseline accuracy is ', round(((my_counter.most_common(1)[0][1])/len(event_log))*100, 2), '%')
# So one would always predict "W_Completeren aanvraag"


Counter({'W_Completeren aanvraag': 42401, 'W_Nabellen offertes': 39869, 'W_Nabellen incomplete dossiers': 17191, 'W_Valideren aanvraag': 14571, 'W_Afhandelen leads': 13644, 'A_SUBMITTED': 10469, 'A_PARTLYSUBMITTED': 10469, 'A_DECLINED': 6022, 'A_PREACCEPTED': 5879, 'O_SELECTED': 5495, 'O_CREATED': 5495, 'O_SENT': 5495, 'A_ACCEPTED': 4053, 'A_FINALIZED': 3980, 'O_CANCELLED': 2685, 'O_SENT_BACK': 2489, 'A_CANCELLED': 2003, 'A_ACTIVATED': 1601, 'A_REGISTERED': 1601, 'A_APPROVED': 1601, 'O_ACCEPTED': 1598, 'O_DECLINED': 548, 'W_Beoordelen fraude': 500, 'W_Wijzigen contractgegevens': 9})
 
The most common event is:  W_Completeren aanvraag  this occurs  42401  times. This is  21.24 % of the events.
 
Therefore the baseline accuracy is  21.24 %


In [7]:
### VISUALIZE THE FREQUENCIES ###

# make a dataframe out of the counter object
my_counter_df = pd.DataFrame.from_dict(my_counter, orient='index').reset_index()

# add the activity type to the dataframe
conditions = [my_counter_df['index'].str.startswith('A', na=False),
              my_counter_df['index'].str.startswith('O', na=False)
              ]
choices = ['Application', 'Offer']

my_counter_df['category'] = np.select(condlist=conditions,
                                      choicelist=choices,
                                      default='Evaluation')

# find the index of the most frequent (in this case that should be "W_Completeren aanvraag")
most_frequent_index = my_counter_df.index[my_counter_df['index'] == my_counter.most_common(1)[0][0]][0]

# color the most frequent red
colors = ['lightslategray',] * len(my_counter_df)
colors[most_frequent_index] = 'crimson'

x = [my_counter_df['category'], my_counter_df['index']]
y= my_counter_df[0]
fig = go.Figure()
fig.add_bar(x=x,y=y, marker={'color':colors} )
fig.update_layout(barmode="relative", height=500, title='Frequency of occurrence activities in the training set.')
fig.show()

In [8]:
### TEST THE BASELINE MODEL ON THE TEST DATA ###

# the baseline model always predicts "W_Completeren aanvraag". Therefore we just need to know what percentage of the events in the test set have this label.

count_most_frequent = len(test_event_log[test_event_log['event concept:name'] == my_counter.most_common(1)[0][0]])
accuracy = count_most_frequent/len(test_event_log)

print('Therefore the baseline accuracy on the test set is ', round(accuracy*100, 2), '%')

Therefore the baseline accuracy on the test set is  23.78 %


In [9]:
### PREPARATION FEATURESPACES ###

# Question: what is the max amount of times an activity happens per instance in the training data set

occurence_activities = event_log.groupby(['case concept:name', 'event concept:name']).size().reset_index(name='counts')
test = occurence_activities.groupby(['event concept:name']).max('counts').reset_index()

# add the activity type to the dataframe
conditions = [test['event concept:name'].str.startswith('A', na=False),
              test['event concept:name'].str.startswith('O', na=False)
              ]
choices = ['Application', 'Offer']

test['category'] = np.select(condlist=conditions,
                             choicelist=choices,
                             default='Evaluation')


# everything that happens at most once should be green, everything that can happen more often grey
colors = test['counts'].apply(lambda x: 'lightslategray' if x>1 else 'green')

# create the plot
x = [test['category'], test['event concept:name']]
y= test['counts']
fig = go.Figure()
fig.add_bar(x=x,y=y, marker={'color':colors} )
fig.update_layout(barmode="relative", height=500, title='Max amount of times an activity occurs in an instance in the training set.')
fig.show()

In [10]:
### BETTER PREDICTIONS ###
# For our models, we will use two different featureSpaces, and two different algorithms.

# FeatureSpace 1: under the markov assumption we look at only the previous event. Therefore the features include:
# - how much did they try to borrow
# - what was the previous event
# - what was the lifecycle of the previous event

# FeatureSpace 2: we assume that the order of the events is irrelevant, we only care if events happened or not. Therefore the features include:
# - how much did they try to borrow
# - what was the previous event
# - what was the lifecycle of the previous event
# - which events have already happened

# We will use both a randomForest and a KNN

In [11]:
### FEATURESPACES ###

# FeatureSpace 1
X_1 = event_log[['case AMOUNT_REQ', 'previous_event', 'previous lifecycle']]

X_1 = pd.get_dummies(X_1)


# FeatureSpace 2
X_2 = event_log[['case AMOUNT_REQ', 'previous_event', 'previous lifecycle', 'A_SUBMITTED', 'A_PARTLYSUBMITTED',
                 'A_PREACCEPTED', 'W_Completeren aanvraag', 'A_ACCEPTED', 'O_SELECTED',
                 'A_FINALIZED', 'O_CREATED', 'O_SENT', 'W_Nabellen offertes',
                 'O_SENT_BACK', 'W_Valideren aanvraag', 'A_ACTIVATED', 'A_REGISTERED',
                 'A_APPROVED', 'O_ACCEPTED', 'O_CANCELLED', 'A_CANCELLED',
                 'W_Afhandelen leads', 'A_DECLINED', 'O_DECLINED',
                 'W_Nabellen incomplete dossiers', 'W_Beoordelen fraude',
                 'W_Wijzigen contractgegevens']]

X_2 = pd.get_dummies(X_2)


# Labels
y = event_log['event concept:name']

In [12]:
### FEATURESPACES - test ###

# FeatureSpace 1
X_1_test = test_event_log[['case AMOUNT_REQ', 'previous_event', 'previous lifecycle']]

X_1_test = pd.get_dummies(X_1_test)
X_1_test['previous_event_W_Wijzigen contractgegevens'] = 0

# FeatureSpace 2
X_2_test = test_event_log[['case AMOUNT_REQ', 'previous_event', 'previous lifecycle', 'A_SUBMITTED', 'A_PARTLYSUBMITTED',
                 'A_PREACCEPTED', 'W_Completeren aanvraag', 'A_ACCEPTED', 'O_SELECTED',
                 'A_FINALIZED', 'O_CREATED', 'O_SENT', 'W_Nabellen offertes',
                 'O_SENT_BACK', 'W_Valideren aanvraag', 'A_ACTIVATED', 'A_REGISTERED',
                 'A_APPROVED', 'O_ACCEPTED', 'O_CANCELLED', 'A_CANCELLED',
                 'W_Afhandelen leads', 'A_DECLINED', 'O_DECLINED',
                 'W_Nabellen incomplete dossiers', 'W_Beoordelen fraude']]

X_2_test = pd.get_dummies(X_2_test)
X_2_test['previous_event_W_Wijzigen contractgegevens'] = 0
X_2_test['W_Wijzigen contractgegevens'] = 0

# Labels
y_test = test_event_log['event concept:name']

### RANDOM FOREST ### => this is optimization, so you don't have to run this anymore


# Since the data is flattened, we can use cross validation.
# To optimize a random forest, one can play around with the depth of the trees.
depths = [1,2,3,4,5,6,7,8,9,10,15,20]
X_1_rf_scores = pd.DataFrame()
X_2_rf_scores = pd.DataFrame()
for depth in depths:
    clf = RandomForestClassifier(max_depth=depth, random_state=0)
    X_1_rf_scores[depth] = cross_val_score(clf, X_1, y, cv=5)
    X_2_rf_scores[depth] = cross_val_score(clf, X_2, y, cv=5)



# plot all the traces for featureSpace 1
fig = go.Figure()

for depth in depths:
    fig.add_trace(go.Scatter(y= X_1_rf_scores[depth], name=depth))
fig.update_yaxes(range=[0,1], title='Accuracy')
fig.update_xaxes(title='5 fold cross validation')
fig.update_layout(title= '5 fold cross validation on random forest on featureSpace X_1, for different tree depths.',
                  legend_title_text='tree depths')
fig.show()



# plot all the traces for featureSpace 2
fig = go.Figure()

for depth in depths:
    fig.add_trace(go.Scatter(y= X_2_rf_scores[depth], name=depth))
fig.update_yaxes(range=[0,1], title='Accuracy')
fig.update_xaxes(title='5 fold cross validation')
fig.update_layout(title= '5 fold cross validation on random forest on featureSpace X_2, for different tree depths.',
                  legend_title_text='tree depths')
fig.show()

### KNN ### => this is optimization, so you don't have to run this anymore


# Since the data is flattened, we can use cross validation.
# To optimize a knn, one can play around with the amount of neighbors.
nr_of_neighbors = [1,3,5,11,21,51]
X_1_knn_scores = pd.DataFrame()
X_2_knn_scores = pd.DataFrame()
for nr in nr_of_neighbors:
    neigh = KNeighborsClassifier(n_neighbors=nr)
    X_1_knn_scores[nr] = cross_val_score(neigh, X_1, y, cv=5)
    X_2_knn_scores[nr] = cross_val_score(neigh, X_2, y, cv=5)


# plot all the traces for featureSpace 1
fig = go.Figure()

for nr in nr_of_neighbors:
    fig.add_trace(go.Scatter(y= X_1_knn_scores[nr], name=nr))

fig.update_yaxes(range=[0,1], title='Accuracy')
fig.update_xaxes(title='5 fold cross validation')
fig.update_layout(title= '5 fold cross validation on knn on featureSpace X_1, for different k.',
                  legend_title_text='k neighbors')
fig.show()



# plot all the traces for featureSpace 2
fig = go.Figure()

for nr in nr_of_neighbors:
    fig.add_trace(go.Scatter(y= X_2_knn_scores[nr], name=nr))

fig.update_yaxes(range=[0,1], title='Accuracy')
fig.update_xaxes(title='5 fold cross validation')
fig.update_layout(title= '5 fold cross validation on knn on featureSpace X_2, for different k.',
                  legend_title_text='k neighbors')
fig.show()

### OPTIMIZED MODELS ### => this is optimization, so you don't have to run this anymore

optimal_depth_X_1 = 15
optimal_depth_X_2 = 15

optimal_k_X_1 = 50
optimal_k_X_2 = 50

# average accuracy
X_1_rf = np.mean(X_1_rf_scores[optimal_depth_X_1])
X_2_rf = np.mean(X_2_rf_scores[optimal_depth_X_2])
X_1_knn = np.mean(X_1_knn_scores[optimal_k_X_1])
X_2_knn = np.mean(X_2_knn_scores[optimal_k_X_2])
baseline = round(((my_counter.most_common(1)[0][1])/len(event_log)), 2)

# make a plot with all values
fig = go.Figure()

fig.add_trace(go.Scatter(x=['X_1', 'X_2'], y= [baseline, baseline], name='Baseline'))
fig.add_trace(go.Scatter(x=['X_1', 'X_2'], y= [X_1_rf, X_2_rf], name='RandomForest'))
fig.add_trace(go.Scatter(x=['X_1', 'X_2'], y= [X_1_knn, X_2_knn], name='KNN'))

fig.update_yaxes(range=[0,1], title='Accuracy')
fig.update_xaxes(title='mean 5 fold cross validation per featureSpace')
fig.update_layout(title= '5 fold cross validation on randomForest and knn on featureSpaces X_1 and X_2.',
                  legend_title_text='algorithm')
fig.show()

In [13]:
### KNN ###

# FeatureSpace 1
neigh_1 = KNeighborsClassifier(n_neighbors=50)
X_1_cv = np.mean(cross_val_score(neigh_1, X_1, y, cv=5))
print('The average 5 fold cross validation accuracy for KNN on FeatureSpace 1 is: ', X_1_cv)
neigh_1.fit(X_1, y)
print('The test accuracy for KNN on FeatureSpace 1 is: ', neigh_1.score(X_1_test, y_test))


# FeatureSpace 2
neigh_2 = KNeighborsClassifier(n_neighbors=50)
X_2_cv = np.mean(cross_val_score(neigh_2, X_2, y, cv=5))
print('The average 5 fold cross validation accuracy for KNN on FeatureSpace 2 is: ', X_2_cv)
neigh_2.fit(X_2, y)
print('The test accuracy for KNN on FeatureSpace 2 is: ', neigh_2.score(X_2_test, y_test))

The average 5 fold cross validation accuracy for KNN on FeatureSpace 1 is:  0.6631457778170983
The test accuracy for KNN on FeatureSpace 1 is:  0.2860966480563745
The average 5 fold cross validation accuracy for KNN on FeatureSpace 2 is:  0.744636064927961
The test accuracy for KNN on FeatureSpace 2 is:  0.4766325826485164


In [14]:
### RandomForest ###

# FeatureSpace 1
rf_1 = RandomForestClassifier(max_depth=15, random_state=0)
X_1_cv_rf = np.mean(cross_val_score(rf_1, X_1, y, cv=5))
print('The average 5 fold cross validation accuracy for RandomForest on FeatureSpace 1 is: ', X_1_cv_rf)
rf_1.fit(X_1, y)
print('The test accuracy for RandomForest on FeatureSpace 1 is: ', rf_1.score(X_1_test, y_test))

# FeatureSpace 2
rf_2 = RandomForestClassifier(max_depth=15, random_state=0)
X_2_cv_rf = np.mean(cross_val_score(rf_2, X_2, y, cv=5))
print('The average 5 fold cross validation accuracy for RandomForest on FeatureSpace 2 is: ', X_2_cv_rf)
rf_2.fit(X_2, y)
print('The test accuracy for RandomForest on FeatureSpace 2 is: ', rf_2.score(X_2_test, y_test))


The average 5 fold cross validation accuracy for RandomForest on FeatureSpace 1 is:  0.7244425191983219
The test accuracy for RandomForest on FeatureSpace 1 is:  0.3917152834410221
The average 5 fold cross validation accuracy for RandomForest on FeatureSpace 2 is:  0.840400039977314
The test accuracy for RandomForest on FeatureSpace 2 is:  0.525249357003952


In [ ]:
### THE NEXT TASK IS TO PREDICT THE TIME UNTIL THE NEXT EVENT ###

In [17]:
### RIDGE REGRESSION ###

### Ridge regression for time prediction ###

# create function mean absolute error
def MAE(Y_actual,Y_Predicted):
    mae = np.mean(np.abs((Y_actual - Y_Predicted)))
    return mae

# create featureSpace
df = pd.read_csv('data/BPI_2012_ridge.csv')
df_train = df[df['split']=='train']
df_test = df[df['split']=='test']
X = df_train[['day_of_week', 'hour', 'month_of_year','position','minutes_since_reg', 'case AMOUNT_REQ', 'START', 'A_SUBMITTED', 'A_PARTLYSUBMITTED', 'A_PREACCEPTED', 'W_Completeren aanvraag', 'A_DECLINED', 'A_ACCEPTED', 'A_FINALIZED', 'O_SELECTED', 'O_CREATED', 'O_SENT', 'W_Nabellen offertes', 'O_CANCELLED', 'W_Afhandelen leads', 'A_CANCELLED', 'O_SENT_BACK', 'W_Valideren aanvraag', 'O_ACCEPTED', 'A_APPROVED', 'A_REGISTERED', 'A_ACTIVATED', 'O_DECLINED', 'W_Nabellen incomplete dossiers', 'W_Beoordelen fraude']]
y = df_train['log_y']

X_test = df_test[['day_of_week', 'hour', 'month_of_year','position','minutes_since_reg', 'case AMOUNT_REQ', 'START', 'A_SUBMITTED', 'A_PARTLYSUBMITTED', 'A_PREACCEPTED', 'W_Completeren aanvraag', 'A_DECLINED', 'A_ACCEPTED', 'A_FINALIZED', 'O_SELECTED', 'O_CREATED', 'O_SENT', 'W_Nabellen offertes', 'O_CANCELLED', 'W_Afhandelen leads', 'A_CANCELLED', 'O_SENT_BACK', 'W_Valideren aanvraag', 'O_ACCEPTED', 'A_APPROVED', 'A_REGISTERED', 'A_ACTIVATED', 'O_DECLINED', 'W_Nabellen incomplete dossiers', 'W_Beoordelen fraude']]
Y_test = df_test['shifted_deltatimes']

# create model
model = Ridge(alpha= 0.03)
model.fit(X, y)

# make predictions
ridge_predict = np.exp(model.predict(X_test))
ridge_MAE = MAE(Y_test,ridge_predict)
r2 = model.score(X_test,df_test['log_y'])

# print results
print(f'MAE: {ridge_MAE/3600} hours')
print(f'R2: {r2}')

MAE: 9.529970803176951 hours
R2: 0.4339549028890677


In [18]:
### RIDGE REGRESSION ###

### Random Forest for time prediction ###

# load train and test set
df_train = pd.read_csv('data/time_rf_2017_train.csv')
df_test = pd.read_csv('data/time_rf_2017_test.csv')

# featureSpace
list_features = ['case LoanGoal', 'case ApplicationType', 'case RequestedAmount', 'event concept:name', 'event EventOrigin', 'event lifecycle:transition', 'hour',
                 'day_of_week', 'Position']
X_train = df_train[list_features]
y_train = df_train['minutes until next']
X_test = df_test[list_features]
y_test = df_test['minutes until next']

# labelEncoder
le = LabelEncoder()

label = le.fit_transform(X_train['case LoanGoal'])
X_train['case LoanGoal'] = label
label = le.fit_transform(X_train['case ApplicationType'])
X_train['case ApplicationType'] = label
label = le.fit_transform(X_train['event concept:name'])
X_train['event concept:name'] =  label
label = le.fit_transform(X_train['day_of_week'])
X_train['day_of_week'] =  label
label = le.fit_transform(X_train['event EventOrigin'])
X_train['event EventOrigin'] =  label
label = le.fit_transform(X_train['event lifecycle:transition'])
X_train['event lifecycle:transition'] =  label
label = le.fit_transform(X_test['case LoanGoal'])
X_test['case LoanGoal'] = label
label = le.fit_transform(X_test['case ApplicationType'])
X_test['case ApplicationType'] =  label
label = le.fit_transform(X_test['event concept:name'])
X_test['event concept:name'] = label
label = le.fit_transform(X_test['day_of_week'])
X_test['day_of_week'] =  label
label = le.fit_transform(X_test['event EventOrigin'])
X_test['event EventOrigin'] =  label
label = le.fit_transform(X_test['event lifecycle:transition'])
X_test['event lifecycle:transition'] =  label

# build model
regressor = RandomForestRegressor(bootstrap=True, max_depth=30, max_features='sqrt', min_samples_leaf=1, min_samples_split=5, n_estimators=700, random_state = 0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

# print error
errors = abs(y_pred - y_test)
print('Mean Absolute Error:', round(np.mean(errors)/60, 2), 'hours')
r2 = r2_score(y_test, y_pred)
print('R2:', r2)

C:\Users\20182960\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\20182960\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\20182960\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

Mean Absolute Error: 72.7 hours
R2: -1.1355295603159292
